In [1]:
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay
from enum import IntEnum
import time

overlay = Overlay('./bram_test.bit')

/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/pl_server/xrt_device.py:59: UserWarning: xbutil failed to run - unable to determine XRT version
  warnings.warn("xbutil failed to run - unable to determine XRT version")


In [2]:
cdma = overlay.axi_cdma_0

In [3]:
input_buffer0 = allocate(shape=(2048*4,), dtype=np.uint32)
output_buffer0 = allocate(shape=(2048*4,), dtype=np.uint32)

for i in range(2048*4):
   input_buffer0[i] = i

print("Input:")
print(input_buffer0)

print()

def getbit(value, order):
    orderVal = 2**order
    tmpbin = value & orderVal
    if (tmpbin):
        return 1
    else:
        return 0

def changebit(value, order, bit):
    if (bit == 1):
        return value | (1 << order)
    else:
        return value & ~(1 << order)

class Cdma(IntEnum):
    CDMACR = 0x0
    CDMASR = 0x4
    SA = 0x18
    DA = 0x20
    BTT = 0x28

def transfer(cdma, src, dst, size):
    # Step 1
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    if (cdmasrIdle != 1):
        print("CDMA is busy..")
        return

    # Step 2
    cdmacr = cdma.read(Cdma.CDMACR)
    cdmacr = changebit(cdmacr, 12, 1) # set IOC_IrqEn
    cdmacr = changebit(cdmacr, 14, 1) # set ERR_IrqEn
    cdma.write(Cdma.CDMACR, cdmacr)

    # Step 3
    cdma.write(Cdma.SA, src)

    # Step 4
    cdma.write(Cdma.DA, dst)

    # Step 5
    cdma.write(Cdma.BTT, size)

    # Step 6
    print("Transferring...")
    cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    while (cdmasrIdle != 1):
        print(".", end="")
        cdma.read(Cdma.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)

    # Step 7-8
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasr = changebit(cdmasr, 12, 1) # clear IOC_Irq
    cdma.write(Cdma.CDMASR, cdmasr)

    print("Transfered " + str(size) + " bytes from " + str(src) + " to " + str(dst))
    print("CDMA Done.")

def reset(cdma):
    cdmacr = cdma.read(Cdma.CDMACR)
    print(type(cdmacr))
    print(cdmacr)
    cdmacr = changebit(cdmacr, 2, 1)
    print(cdmacr)
    cdma.write(Cdma.CDMACR, cdmacr)

print("==== Test CDMA ====")

start_time = time.time()
transfer(cdma, input_buffer0.physical_address, 0xE000_0000, 2048*4*4)
transfer(cdma, 0xE000_0000,output_buffer0.physical_address, 2048*4*4)

end_time = time.time()
print(output_buffer0)
print("%s seconds" % (end_time - start_time))
print()




Input:
[   0    1    2 ... 8189 8190 8191]

==== Test CDMA ====
Transferring...
Transfered 32768 bytes from 965148672 to 3758096384
CDMA Done.
Transferring...
Transfered 32768 bytes from 3758096384 to 965181440
CDMA Done.
[   0    1    2 ... 8189 8190 8191]
0.0013637542724609375 seconds

